# Python functies voor Odata4
Functies voor ophalen, inspecteren en samenvoegen van data van CBS

## Meer info

- https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/snelstartgids-odata-v4
- https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/metadata-odata-v4
- Ook code voor R beschikbaar.

# Definieren functies

In [1]:
import pandas as pd
import requests
import re

In [2]:
import pandas as pd
import requests

def get_odata(target_url):
    
    """"
    De functie gebruikt een API genaamd OData om data van het CBS op te halen.
    De data wordt in stukken opgehaald en in een pandas dataframe gezet.
    De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
    De code van de tabel die je zoekt vindt je via Statline.
    Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
    """
    
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

De functie gebruikt een API genaamd OData om data van het CBS op te halen.
De data wordt in stukken opgehaald en in een pandas dataframe gezet.
De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
De code van de tabel die je zoekt vindt je via Statline.
Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
Deze API is hierarchisch. 

In [3]:
def get_observations(table_url, url_filter = ""):
    
    """Haal de tabel met metingen op. Filteren om minder of specifiekere data op te vragen is mogelijk as volgt:
    url_filter = ?$filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'
    Door de filteren op deze kolommen kun je een plaats (land, gemeente, wijk of buurt) en dan een soort meting kiezen.
    De website van het CBS vermeldt niet hoe je op meerdere waarden in één kolom filtert. 
    Die mogelijkheid bestaat waarschijnlijk wel. Je hoeft niet beide kolommen te gebruiken voor het filter.
    """
    
    if url_filter == "":
        target_url = table_url + "/Observations"
    elif "?$filter=" in url_filter:
        target_url = table_url + "/Observations" + url_filter
    else:
        print("WAARSCHUWING! FILTER NIET GOED GEFORMATTEERD. VERGELIJK MET VOORBEELDEN OF GA NAAR ")
        print("\n https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4")
        print("\n http://docs.oasis-open.org/odata/odata/v4.0/errata03/os/complete/part2-url-conventions/odata-v4.0-errata03-os-part2-url-conventions-complete.html#_Toc453752358")
        print("\n sectie 5.1.1.5")
        #return None
        pass
    
    data = get_odata(target_url)
    print(data.head())
    
    return(data)

url_voorbeeld = "https://odata4.cbs.nl/CBS/83765NED"

# Wat is de structuur van de data?

In [4]:
data_structuur = get_odata(url_voorbeeld)
print(data_structuur)
data_Mcodes = get_odata(url_voorbeeld+"/MeasureCodes")
print(data_Mcodes.columns)

        kind                   name                    url
0  EntitySet          MeasureGroups          MeasureGroups
1  EntitySet           MeasureCodes           MeasureCodes
2  EntitySet             Dimensions             Dimensions
3  EntitySet  WijkenEnBuurtenGroups  WijkenEnBuurtenGroups
4  EntitySet   WijkenEnBuurtenCodes   WijkenEnBuurtenCodes
5  EntitySet           Observations           Observations
6  Singleton             Properties             Properties
Index(['DataType', 'Decimals', 'Description', 'Format', 'Identifier', 'Index',
       'MeasureGroupId', 'PresentationType', 'Title', 'Unit'],
      dtype='object')


In [50]:
# Maak een csv van de codes om ze in Excel goed te bekijken
data_Mcodes.to_csv("MeasureCodes.csv", sep=";", na_rep="None")
data_Mcodes[['Title','Description','Identifier','MeasureGroupId']].to_csv("features_alles.csv", sep=";", na_rep="None")
data_Mcodes
#data_Mcodes['Title'].values
#data_Mcodes["Identifier"]

,DataType,Decimals,Description,Format,Identifier,Index,MeasureGroupId,PresentationType,Title,Unit
0,Long,0,,None,T001036,7,M000352,Absolute,Aantal inwoners,aantal
1,Long,0,,None,3000,9,T001038,Absolute,Mannen,aantal
2,Long,0,,None,4000,10,T001038,Absolute,Vrouwen,aantal
3,Long,0,Aantal inwoners dat op 1 januari 0 tot 15 jaar...,None,10680,12,10000,Absolute,0 tot 15 jaar,aantal
4,Long,0,Aantal inwoners dat op 1 januari 15 tot 25 jaa...,None,53050,13,10000,Absolute,15 tot 25 jaar,aantal
5,Long,0,Aantal inwoners dat op 1 januari 25 tot 45 jaa...,None,53310,14,10000,Absolute,25 tot 45 jaar,aantal
6,Long,0,Aantal inwoners dat op 1 januari 45 tot 65 jaa...,None,53715,15,10000,Absolute,45 tot 65 jaar,aantal
7,Long,0,Aantal inwoners dat op 1 januari 65 jaar of ou...,None,80200,16,10000,Absolute,65 jaar of ouder,aantal
8,Long,0,Het aantal inwoners dat op 1 januari ongehuwd ...,None,1010,18,T001019,Absolute,Ongehuwd,aantal
9,Long,0,Het aantal inwoners dat op 1 januari gehuwd is...,None,1020,19,T001019,Absolute,Gehuwd,aantal


In [20]:
# het voorbeeld van de website van CBS:
#table_url = "https://odata4.cbs.nl/CBS/83765NED"

#target_url = table_url + "/Observations"
#data = get_odata(target_url)
#print(data.head())

   Id  Measure       Value ValueAttribute WijkenEnBuurten
0   0  T001036  17081507.0           None            NL00
1   1     3000   8475102.0           None            NL00
2   2     4000   8606405.0           None            NL00
3   3    10680   2781768.0           None            NL00
4   4    53050   2101648.0           None            NL00


In [3]:
#tabel 2019
#lastyear_url = "https://odata4.cbs.nl/CBS/84583NED"

,name,url
0,TableInfos,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
1,UntypedDataSet,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
2,TypedDataSet,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
3,DataProperties,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
4,CategoryGroups,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
5,WijkenEnBuurten,https://opendata.cbs.nl/oDataAPI/OData/84583NE...


## Filteren van query
Het filteren van de data maakt het downloaden sneller.
Het filteren van 'Observations' data kan door code van dit format achter de url te plakken:

**?$filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'**

De code uit de kolom WijkenEnBuurten kun je vinden met 

**get_odata(table_url + "/WijkenEnBuurtenCodes")**

De 'Title' kolom van deze tabel bevat de namen van wijken, zodat je kan zoeken met str.find, <>.str.contains of Regex.
Zoals wel vaken met tektskolommen moet je dan vertrouwen op de volledigheid en consistentie.
Achteraf controleren of je wel de juiste weijken hebt s dus wel aangeraden.
De kolom WijkenenBuurten bevat zowel landen, gemeenten, wijken als buurten.
Aan het voorvoegsel van twee letters kun je zien met welke soort regio je te maken hebt.

Zie https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4 voor meer uitleg.

In [10]:
table_test = get_observations(url_voorbeeld, url_filter="?$filter=contains(WijkenEnBuurten,'GM')")
#table_test = get_observations(url_voorbeeld)

    Id  Measure    Value ValueAttribute WijkenEnBuurten
0  103  T001036  25286.0           None          GM1680
1  104     3000  12603.0           None          GM1680
2  105     4000  12683.0           None          GM1680
3  106    10680   3572.0           None          GM1680
4  107    53050   2558.0           None          GM1680


In [11]:
print(table_test.columns)
print(table_test.head(30))
# Kolom ValueAttribute heeft geen waarden

Index(['Id', 'Measure', 'Value', 'ValueAttribute', 'WijkenEnBuurten'], dtype='object')
     Id    Measure    Value ValueAttribute WijkenEnBuurten
0   103    T001036  25286.0           None          GM1680
1   104       3000  12603.0           None          GM1680
2   105       4000  12683.0           None          GM1680
3   106      10680   3572.0           None          GM1680
4   107      53050   2558.0           None          GM1680
5   108      53310   4383.0           None          GM1680
6   109      53715   8467.0           None          GM1680
7   110      80200   6306.0           None          GM1680
8   111       1010   9951.0           None          GM1680
9   112       1020  11895.0           None          GM1680
10  113       1080   1793.0           None          GM1680
11  114       1050   1647.0           None          GM1680
12  115    2012655   1056.0           None          GM1680
13  116  2012657_2    528.0           None          GM1680
14  117    H008673     12.0 

In [13]:
# Deze tabel bevat alle gemeente, maar GEEN info over wijken en buurten
data_gemeenten = get_odata(url_voorbeeld+"/WijkenEnBuurtenGroups")
print(data_gemeenten.size)
print(data_gemeenten.columns)
print(data_gemeenten.head(10))
# ParentId geeft voor wijken de buurt aan, voor wijken de plaatsnaam, voorplaatsnamen de gemeente etc.

1960
Index(['Description', 'Id', 'Index', 'ParentId', 'Title'], dtype='object')
  Description      Id  Index ParentId                           Title
0        None    WBGM      0     None  Wijken en buurten per gemeente
1        None  GM1680      1     WBGM                     Aa en Hunze
2        None  GM0738      2     WBGM                         Aalburg
3        None  GM0358      3     WBGM                        Aalsmeer
4        None  GM0197      4     WBGM                          Aalten
5        None  GM0059      5     WBGM                   Achtkarspelen
6        None  GM0482      6     WBGM                    Alblasserdam
7        None  GM0613      7     WBGM                   Albrandswaard
8        None  GM0361      8     WBGM                         Alkmaar
9        None  GM0141      9     WBGM                          Almelo


In [14]:
filter_gemeente = data_gemeenten['Title'] == 'Oss'
print(data_gemeenten[filter_gemeente])
selectie_gem = data_gemeenten[filter_gemeente]['Id'].values[0]
selectie_gem

    Description      Id  Index ParentId Title
254        None  GM0828    254     WBGM   Oss


'GM0828'

In [15]:
# Deze tabel bevat codes van wijken en buurten en toont bij welke gemeente ze horen. De gemeenten staan er ook in.
data_geocodes = get_odata(url_voorbeeld+"/WijkenEnBuurtenCodes")
print(data_geocodes.size)
print(data_geocodes.columns)
print(data_geocodes.head(10))
# Ik denk dat DimensionGroupId te maken heeft met hierarchische indeling maar niet hetzelfde is als parentId.
# DetailRegionCode is hetzelfde als Identifier

100002
Index(['Description', 'DetailRegionCode', 'DimensionGroupId', 'Identifier',
       'Index', 'Title'],
      dtype='object')
  Description DetailRegionCode DimensionGroupId  Identifier  Index  \
0                         None               NL        NL00      1   
1                       GM1680               GM      GM1680      2   
2                     WK168000           GM1680    WK168000      3   
3                   BU16800000           GM1680  BU16800000      4   
4                   BU16800009           GM1680  BU16800009      5   
5                     WK168001           GM1680    WK168001      6   
6                   BU16800100           GM1680  BU16800100      7   
7                   BU16800109           GM1680  BU16800109      8   
8                     WK168002           GM1680    WK168002      9   
9                   BU16800200           GM1680  BU16800200     10   

                     Title  
0                Nederland  
1              Aa en Hunze  
2          

In [26]:
# Gebruik van Regular Expressions sterk aanbevolen om verschil met hele woorden te zien,
# Maakt implementatie wel iets ingewikkelder.
filter_BU = data_geocodes['Identifier'].str.contains("BU")
filter_WK = data_geocodes['Identifier'].str.contains("WK")

filter_BUWKopgem = data_geocodes['DimensionGroupId'] == selectie_gem
#print(data_geocodes[filter_BUWKopgem])
selectie_WK = data_geocodes[filter_WK & filter_BUWKopgem]["Identifier"]
print(selectie_WK)
#selectie_BU = data_geocodes[filter_BU & filter_BUWKopgem]["Identifier"]
#print(selectie_BU)

11353    WK082800
11357    WK082801
11365    WK082802
11369    WK082803
11378    WK082804
11384    WK082805
11389    WK082806
11400    WK082807
11411    WK082808
11416    WK082809
11422    WK082810
11428    WK082811
11432    WK082812
11436    WK082813
11440    WK082814
11446    WK082815
11450    WK082816
11454    WK082817
11460    WK082818
11463    WK082819
11468    WK082820
11471    WK082821
11477    WK082822
Name: Identifier, dtype: object


# Selecteren features en samenvoegen data

In [53]:
def feature_select(csv_file):
    """Inlezen van csv gebaseerd op de MeasureCodes met alleen de rijen van features die je wilt ophalen.
    Het verminderen van de hoeveelheid data maakt het ophalen van data sneller."""
    
    df = pd.read_csv(csv_file, sep=";")
    return df

#feature_select("features_test2.csv")
feature_selectie=feature_select("features_test2.csv")['Identifier']

,Unnamed: 0,Title,Description,Identifier,MeasureGroupId
0,0,Aantal inwoners,NaN,T001036,M000352
1,1,Mannen,NaN,3000,T001038
2,2,Vrouwen,NaN,4000,T001038
3,3,0 tot 15 jaar,Aantal inwoners dat op 1 januari 0 tot 15 jaar...,10680,10000
4,4,15 tot 25 jaar,Aantal inwoners dat op 1 januari 15 tot 25 jaa...,53050,10000
5,5,25 tot 45 jaar,Aantal inwoners dat op 1 januari 25 tot 45 jaa...,53310,10000
6,6,45 tot 65 jaar,Aantal inwoners dat op 1 januari 45 tot 65 jaa...,53715,10000
7,7,65 jaar of ouder,Aantal inwoners dat op 1 januari 65 jaar of ou...,80200,10000
8,8,Ongehuwd,Het aantal inwoners dat op 1 januari ongehuwd ...,1010,T001019
9,9,Gehuwd,Het aantal inwoners dat op 1 januari gehuwd is...,1020,T001019


In [29]:
def feature_reader(text_file):
    """Snel inlezen van een leesbaar tekstbestand met namen van features.
    Niet meer gebruiken, want het bevat de codes niet en is dus niet bruikbaar voor het bewerken van dataframes."""
    features = open(text_file,"r")
    features = features.readlines()
    features2 = ""
    for line in features:
        features2 += line
    # Verwijder \n, scheid op , en verwijder spaties aan begin
    features2 = re.sub("\n", "", features2)
    features2 = re.split(",", features2)
    features = []
    for feat in features2:
        features.append(re.sub("^ ","", feat))
        
    return features

#features = feature_reader("features_test1.txt")
#features

In [30]:
# gebruik 
url_test = "?$filter=WijkenEnBuurten eq '"+selectie_gem+"'"
# Alleen op deze manier kan je een ID uit de geografische tabel in het url-filter plaatsen. Ook de spaties tellen.)
data_observations = get_observations(url_voorbeeld, url_filter=url_test)
data_observations.head()

        Id  Measure   Value ValueAttribute WijkenEnBuurten
0  1169358  CRI3000     4.0           None          GM0828
1  1169357  CRI2000     5.0           None          GM0828
2  1169356  CRI1100     3.0           None          GM0828
3  1169355   ST0003  1360.0           None          GM0828
4  1169354   ST0001     3.0           None          GM0828


,Id,Measure,Value,ValueAttribute,WijkenEnBuurten
0,1169358,CRI3000,4.0,None,GM0828
1,1169357,CRI2000,5.0,None,GM0828
2,1169356,CRI1100,3.0,None,GM0828
3,1169355,ST0003,1360.0,None,GM0828
4,1169354,ST0001,3.0,None,GM0828


In [31]:
data_observations["Measure"]

0        CRI3000
1        CRI2000
2        CRI1100
3         ST0003
4         ST0001
5        A047040
6        A047044
7      T001455_2
8        D000263
9        D000045
10       D000029
11       D000025
12       D000028
13       A018944
14     A018943_5
15       M000368
16       M002179
17       A019276
18         72003
19         40001
20     A018943_2
21        300014
22        300010
23        300009
24        383105
25        300005
26        300003
27        301000
28     M000200_2
29       D000193
         ...    
71       M000297
72       M000100
73       M000114
74       1016030
75       1016040
76       1050015
77     1050010_2
78     M000179_2
79     M000179_1
80     M000173_2
81     M000173_1
82       A008187
83       H008766
84       H008751
85       H007119
86       H008673
87     2012657_2
88       2012655
89          1050
90          1080
91          1020
92          1010
93         80200
94         53715
95         53310
96         53050
97         10680
98          40

In [47]:
test2 = table_test.pivot(index="Id", columns="Measure", values="Value") # Zelfs voor hele dataset snel te doen
test2 = test2.merge(table_test[["Id","WijkenEnBuurten"]], how="left", left_index=True, right_on="Id").drop("Id", axis=1)
#test2.head(20)
#test2.groupby("WijkenEnBuurten").first()
# Deze tabel heeft het gewenste format. 

In [43]:
def formatteer_tabel(df):
    """Formatteert een tabel met observaties naar een formaat met één kolom voor elke meting en één rij per plaats.
    De codes voor plaatsen worden de index, de kolommen krijgen als naam de Identifier van de measure.
    Dat betekent dat extra informatie over de hierarchie van measures of plaatsen nog moet worden toegevoegd.
    Deze functie verwijdert dubbele waarden en vult NaN's niet in."""
    
    df2 = df.pivot(index="Id", columns="Measure", values="Value")
    df2 = df2.merge(df[["Id","WijkenEnBuurten"]], how="left", left_index=True, right_on="Id").drop("Id", axis=1)
    df2 = df2.groupby("WijkenEnBuurten").first()
    return df2

formatteer_tabel(table_test)

,1010,1014800_1,1014800_2,1014800_3,1014850_2,1014850_3,1014850_4,1016030,1016040,1020,...,ZW10320_2,ZW10340,ZW25805_1,ZW25805_2,ZW25806_1,ZW25806_2,ZW25807,ZW25808,ZW25810_1,ZW25810_2
WijkenEnBuurten,,,,,,,,,,,,,,,,,,,,,
GM0003,5075.0,48.0,2830.0,1630.0,51.0,2140.0,1220.0,1733.0,1800.0,4997.0,...,1960.0,28.0,2560.0,1360.0,2650.0,1550.0,15.0,85.0,1790.0,960.0
GM0005,4599.0,68.0,2830.0,1680.0,31.0,2070.0,1280.0,1580.0,1526.0,4720.0,...,2020.0,14.0,2530.0,1380.0,2520.0,1510.0,14.0,86.0,1690.0,890.0
GM0007,3259.0,71.0,3520.0,2040.0,28.0,2370.0,1620.0,1179.0,1606.0,4294.0,...,2130.0,6.0,2230.0,1470.0,2400.0,1630.0,6.0,94.0,1810.0,1080.0
GM0009,3231.0,77.0,2890.0,1730.0,23.0,2100.0,1340.0,1176.0,987.0,3137.0,...,2000.0,9.0,2530.0,1440.0,2570.0,1610.0,15.0,85.0,1740.0,780.0
GM0010,10264.0,62.0,2890.0,1670.0,37.0,2160.0,1310.0,3585.0,3815.0,10894.0,...,2000.0,22.0,2650.0,1400.0,2660.0,1570.0,9.0,91.0,1750.0,1020.0
GM0014,137073.0,38.0,2720.0,1360.0,61.0,2010.0,1100.0,21628.0,25353.0,44753.0,...,2230.0,64.0,2730.0,1320.0,2910.0,1530.0,13.0,87.0,1910.0,1050.0
GM0015,5660.0,72.0,3480.0,1870.0,26.0,2210.0,1470.0,2032.0,1528.0,5322.0,...,2030.0,8.0,2290.0,1380.0,2500.0,1480.0,15.0,85.0,1750.0,940.0
GM0017,8530.0,69.0,3330.0,2100.0,31.0,2130.0,1380.0,2821.0,2753.0,8250.0,...,3020.0,23.0,2670.0,1510.0,2790.0,1680.0,9.0,91.0,1900.0,1110.0
GM0018,15280.0,56.0,3080.0,1710.0,43.0,2240.0,1210.0,5094.0,4842.0,13684.0,...,2260.0,30.0,2700.0,1380.0,2850.0,1600.0,15.0,85.0,1980.0,970.0


# To do: 

- Maak dictionary om Measure-codes makkelijk te vertalen.
- Test manieren om data op te vragen op basis van adressen en een lijst features
- Fix Odata3 parser met slim gekozen filters en dezelfde functies als OData4
- Analyseer en bewerk NaN's. Veel algoritmen van SciKit kunnen niet overweg met NaN.

Dan zouden alle gewenste functies klaar moeten zijn en kan het notebook worden opgeschoont of omgezet naar .py
De hierarchische indeling van locaties moet nog wel toegevoegd worden maar omdat daar andere bestanden voor nodig zijn
is het netjes om in een ander notebook verder te gaan.